In [1]:
# do a hacky import since brainaudio requires python 3.12 and the .wfst env requires an older
# python version 
from wfst_utils import build_lm_decoder, lm_decode, arrange_logits, _cer_and_wer
import sys
import numpy as np
import re
import pickle
import pandas as pd

In [2]:
load_lm = True

In [3]:
transcripts_24_cleaned = pd.read_pickle("/home/ebrahim/data2/brain2text/b2t_24/transcripts_val_cleaned.pkl")

In [4]:
blank_penalty = np.log(7)

In [5]:
lmDir = "/home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test"
acoustic_scale = 0.5 
nbest = 100
return_n_best = True
rescore = True

if load_lm and 'ngramDecoder' not in globals(): 
    
    ngramDecoder = build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")

loaded LM


I0203 01:51:42.584389 21645 brain_speech_decoder.h:52] Reading fst /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/TLG.fst
I0203 01:53:45.067565 21645 brain_speech_decoder.h:58] Reading lm fst /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/G.fst
I0203 01:54:11.672883 21645 brain_speech_decoder.h:70] Reading rescore fst /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/G_no_prune.fst
I0203 02:00:42.793962 21645 brain_speech_decoder.h:81] Reading symbol table /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/words.txt


In [10]:
logits_all = np.load("/home/ebrahim/data2/brain2text/b2t_24/logits/bidirectional_gru_all/bidirectional_gru_seed_0/logits_test.npz")
print(len(logits_all))

1200


In [11]:
trial_indices = np.arange(1200)

In [12]:
nbest_outputs = []

for trial_idx in trial_indices:
    
    if trial_idx % 100 == 0:
        
        print(f"Decoding trial {trial_idx}...")
    
    logits = logits_all[f'arr_{trial_idx}']
        
    rearranged_logits = arrange_logits(logits)
    
    nbest = lm_decode(
                    ngramDecoder,
                    rearranged_logits[0],
                    blankPenalty=blank_penalty,
                    returnNBest=return_n_best,
                    rescore=rescore,
                )
    
    nbest_outputs.append(nbest)
    

Decoding trial 0...
Decoding trial 100...
Decoding trial 200...
Decoding trial 300...
Decoding trial 400...
Decoding trial 500...
Decoding trial 600...
Decoding trial 700...
Decoding trial 800...
Decoding trial 900...
Decoding trial 1000...
Decoding trial 1100...


In [13]:
nbest_save_path = "/home/ebrahim/data2/brain2text/b2t_24/model_outputs/bidirectional_gru_test/bidirectional_gru_seed_0_nbest_5gram.pkl"

In [14]:
with open(nbest_save_path, "wb") as f:
    pickle.dump(nbest_outputs, f)